# Prueba de consultas

En esta seccion probamos las consultas

In [38]:
import os
import pandas as pd
import fastapi
from platform import python_version
print(python_version()) # python 3.7.16

3.10.12


In [39]:
data_mvp_funciones = pd.read_csv(os.path.join('3_output','data_mvp_final_funciones.csv'),index_col=0).convert_dtypes()
data_mvp_director = pd.read_csv(os.path.join('3_output','data_mvp_final_funciones_exitodir.csv'),index_col=0).convert_dtypes()
data_mvp_recomendacion = pd.read_csv(os.path.join('3_output','data_mvp_ml_recomend_indexed.csv'),index_col=[0,1]).convert_dtypes()

#app = FastAPI()

Instruccion:

def peliculas_idioma( Idioma: str ): Se ingresa un idioma (como están escritos en el dataset, no hay que traducirlos!). Debe devolver la cantidad de películas producidas en ese idioma.

Ejemplo de retorno: X cantidad de películas fueron estrenadas en idioma

In [40]:
#@app.get("/peliculas_idioma/{idioma}")
# definimos
def pelicula_idioma(idioma: str):
    cantidad = data_mvp_funciones['pelicula_id'].loc[
        data_mvp_funciones['original_language'].isin([idioma])].nunique()
    return f"{cantidad} películas fueron estrenadas en idioma: {idioma}"

# test
pelicula_idioma('en')

'32189 películas fueron estrenadas en idioma: en'

Instruccion

def peliculas_duracion( Pelicula: str ): Se ingresa una pelicula. Debe devolver la duracion y el año.

Ejemplo de retorno: X . Duración: x. Año: xx

In [41]:
#@app.get("/peliculas_duracion/{pelicula}")
# definimos
def pelicula_duracion(pelicula: str):
    pelicula_title = str(pelicula)
    duracion = data_mvp_funciones.loc[data_mvp_funciones['title'].str.contains(pelicula_title), 'runtime'].iloc[0]
    release_year = data_mvp_funciones.loc[data_mvp_funciones['title'].str.contains(pelicula_title), 'release_year'].iloc[0]
    return f"{pelicula_title}. Duración: {round(duracion)} min; año: {release_year}"

# test
pelicula_duracion('Toy Story')

'Toy Story. Duración: 81 min; año: 1995'

Instruccion:

def franquicia( Franquicia: str ): Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio

Ejemplo de retorno: La franquicia X posee X peliculas, una ganancia total de x y una ganancia promedio de xx

In [42]:
#@app.get("/franquicia/{franquicia}")
# definimos
def franquicia(franquicia: str):
    franquicia_str=str(franquicia)
    franquicia_subset = data_mvp_funciones.loc[
        data_mvp_funciones['franquicia'].str.contains(franquicia_str)]
    cantidad = franquicia_subset['pelicula_id'].nunique()
    ganancia = franquicia_subset['revenue'].sum()
    ganancia_res = ('{:,}'.format(round(ganancia)))
    promedio = franquicia_subset['revenue'].mean()
    promedio_res = ('{:,}'.format(round(promedio)))
    return f"La franquicia {franquicia_str} posee {cantidad} película(s), una ganancia total de USD${ganancia_res}, y una ganancia promedio de USD${promedio_res} por pelicula."

# test
franquicia('Toy Story Collection')

'La franquicia Toy Story Collection posee 3 película(s), una ganancia total de USD$3,004,860,308, y una ganancia promedio de USD$751,215,077 por pelicula.'

Instruccion:

def peliculas_pais( Pais: str ): Se ingresa un país (como están escritos en el dataset, no hay que traducirlos!), retornando la cantidad de peliculas producidas en el mismo.

Ejemplo de retorno: Se produjeron X películas en el país X

In [43]:
#@app.get("/peliculas_pais/{pais}")
# definimos
def pelicula_pais(pais: str):
    pais_str = str(pais)
    cantidad = data_mvp_funciones['pelicula_id'].loc[
       data_mvp_funciones['pais_name'].str.contains(pais_str)].nunique()
    return f"Se produjeron {cantidad} películas en el país {pais}"
# test
pelicula_pais('United States of America')

'Se produjeron 2570 películas en el país United States of America'

Instruccion:

def productoras_exitosas( Productora: str ): Se ingresa la productora, entregandote el revunue total y la cantidad de peliculas que realizo.

Ejemplo de retorno: La productora X ha tenido un revenue de x

In [44]:
#@app.get("/productoras_exitosas/{productora}")
# definimos
def productora_exitosa(productora: str):
    productora_subset = data_mvp_funciones.loc[
        data_mvp_funciones['productora'].str.contains(productora)]
    revenue = productora_subset['revenue'].sum()
    revenue_res = ('{:,}'.format(round(revenue)))
    cantidad = productora_subset['pelicula_id'].nunique()
    return f"La productora {productora} ha tenido un revenue total de USD${revenue_res}, y realizo un total de {cantidad} de pelicula(s)"

# test
productora_exitosa('Pixar Animation Studios')

'La productora Pixar Animation Studios ha tenido un revenue total de USD$9,267,027,021, y realizo un total de 12 de pelicula(s)'

Instruccion

def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma, en formato lista.

In [45]:
data_mvp_director.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18614 entries, 0 to 18613
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   director          18614 non-null  string 
 1   director_revenue  18614 non-null  Int64  
 2   director_budget   18614 non-null  Int64  
 3   director_return   18614 non-null  Float64
 4   pelicula_id       18614 non-null  Int64  
 5   title             18614 non-null  string 
 6   release_date      18614 non-null  string 
 7   release_year      18614 non-null  Int64  
 8   budget            18614 non-null  Int64  
 9   revenue           18614 non-null  Int64  
 10  return            18614 non-null  Float64
dtypes: Float64(2), Int64(6), string(3)
memory usage: 1.8 MB


In [46]:
#@app.get("/get_director/{director}")
# definimos
def director_exitoso(director: str):
    director_subset = data_mvp_director.loc[
        data_mvp_director['director'].str.contains(director)]
    director_return = director_subset['director_return'].drop_duplicates()
    director_return_res = round(float(director_return.iloc[0]),2)
    columns_peliculas=['pelicula_id','title','release_date','budget','revenue','return']
    director_peliculas = director_subset[columns_peliculas].drop_duplicates()
    peliculas = []
    for _, pelicula in director_peliculas.iterrows():
        pelicula_info = {
            'nombre': pelicula['title'],
            'fecha_lanzamiento': pelicula['release_date'],
            'costo (USD)': ('{:,}'.format(round(pelicula['budget']))),
            'ganancia (USD)': ('{:,}'.format(round(pelicula['revenue']))),
            'retorno (ratio)': round(pelicula['return'],2)
        }
        peliculas.append(pelicula_info)
    return {
        'director': director,
        'retorno total': director_return_res,
        'peliculas': peliculas }
# test
director_exitoso('Woody Allen')

{'director': 'Woody Allen',
 'retorno total': 3.02,
 'peliculas': [{'nombre': 'Mighty Aphrodite',
   'fecha_lanzamiento': '1995-09-13',
   'costo (USD)': '15,000,000',
   'ganancia (USD)': '6,700,000',
   'retorno (ratio)': 0.45},
  {'nombre': 'Bullets Over Broadway',
   'fecha_lanzamiento': '1994-10-14',
   'costo (USD)': '0',
   'ganancia (USD)': '0',
   'retorno (ratio)': 0.0},
  {'nombre': 'Manhattan Murder Mystery',
   'fecha_lanzamiento': '1993-08-18',
   'costo (USD)': '13,500,000',
   'ganancia (USD)': '11,285,588',
   'retorno (ratio)': 0.84},
  {'nombre': 'Everyone Says I Love You',
   'fecha_lanzamiento': '1996-12-06',
   'costo (USD)': '20,000,000',
   'ganancia (USD)': '0',
   'retorno (ratio)': 0.0},
  {'nombre': 'Sleeper',
   'fecha_lanzamiento': '1973-12-17',
   'costo (USD)': '2,000,000',
   'ganancia (USD)': '18,344,729',
   'retorno (ratio)': 9.17},
  {'nombre': 'Bananas',
   'fecha_lanzamiento': '1971-04-28',
   'costo (USD)': '2,000,000',
   'ganancia (USD)': '11,8

Instruccion:

def recomendacion( titulo ): Se ingresa el nombre de una película y te recomienda las similares en una lista de 5 valores.

In [47]:
data_mvp_recomendacion.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 179170 entries, (1.0, 1.0) to (4.0, 5.0)
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   pelicula_id   179170 non-null  Int64  
 1   cos_smltd     179170 non-null  Float64
 2   vecino_id     179170 non-null  Int64  
 3   title         179170 non-null  string 
 4   vecino_title  179170 non-null  string 
dtypes: Float64(1), Int64(2), string(2)
memory usage: 7.7 MB


In [48]:
#@app.get("/recomendacion/{titulo_pelicula}")
# definimos
def recomendacion_pelicula(titulo_pelicula: str):
    recomendacion_subset = data_mvp_recomendacion.loc[
        data_mvp_recomendacion['title'].isin([titulo_pelicula])]
    pelicula_vecinos = recomendacion_subset[['vecino_title']].reset_index().drop_duplicates()
    pelicula_recomend = list(pelicula_vecinos['vecino_title'])
    return {
        'pelicula': titulo_pelicula,
        'recomendaciones': pelicula_recomend}
# test
recomendacion_pelicula('Toy Story')

{'pelicula': 'Toy Story',
 'recomendaciones': ['Toy Story 2',
  'Totally Fucked Up',
  "City Slickers II: The Legend of Curly's Gold",
  'Halloweentown',
  'Cheech & Chong Get Out of My Room']}